Noemie M.-L. Eustachon - July 2023

Finnley Howald - 2024

Code to fit RL models to behavioural data

**Recommended read:**

Robert C Wilson, Anne GE Collins (2019) Ten simple rules for the computational modeling of behavioral data eLife 8:e49547

https://doi.org/10.7554/eLife.49547




**Import Statements**

In [1]:
import csv
import pandas as pd
from pathlib import Path
import re
import itertools
import numpy as np
import matplotlib.pyplot as plt
import scipy
from scipy.special import softmax, log_softmax
from scipy.optimize import minimize

# **1.0 Data Preprocesing**

## **1.1 Compile All Trial Data from Cohort 5 to one Dataframe**

In [2]:
def short_csv_to_df(file_path):
    '''
    Converts a CSV file with the same format as files ending with "(short)" and converts it to a DataFrame.
    
    Parameters:
        file_path (Path): The path of the CSV file to convert to a DataFrame.
    
    Returns:
        df (DataFrame): A DataFrame representing the important lines from the CSV file.
    '''
    csv_list = []
    column_titles = ["Block", "Tone", "ToneTime", "Unused", "Prob", "Choice", "Reward", "Latency", "Lick", "OutofTotal"]

    with open(file_path, mode='r') as file:
        csv_reader = csv.reader(file)
        first_trial_start = False

        for index, row in enumerate(csv_reader):
            if len(row) > 1 and row[1].strip().isdigit() and int(row[1].strip()) >= 1:
                first_trial_start = True

            if first_trial_start:
                if len(row) >= len(column_titles):
                    csv_list.append(row[:len(column_titles)])
                else:
                    # Pad the row with empty strings if it's shorter than expected
                    csv_list.append(row + [''] * (len(column_titles) - len(row)))

    df = pd.DataFrame(csv_list, columns=column_titles)
    return df

def long_csv_to_df(file_path):
    '''
    Converts a CSV file with the same format as files ending with "(long)" and converts it to a DataFrame.
    
    Parameters:
        file_path (Path): The path of the CSV file to convert to a DataFrame.
    
    Returns:
        df (DataFrame): A DataFrame representing the important lines from the CSV file.
    '''
    csv_list = []
    column_titles = [
        "Trial", "Tone", "Total Licks", "Lick Side Number", "Time", "ToneTime", "Latency", "Decision",
        "Reward", "Num of No Decision", "Num of Consecutive No Decision", "Block", "Left Prob", "Right Prob"
    ]

    with open(file_path, mode='r') as file:
        csv_reader = csv.reader(file)
        for index, row in enumerate(csv_reader):
            if index >= 3:
                if len(row) > 1 and row[1].strip():
                    # Ensure the row has the correct number of columns
                    row = row[:len(column_titles)] + [''] * (len(column_titles) - len(row))
                    csv_list.append(row)

    df = pd.DataFrame(csv_list, columns=column_titles)

    # No need to standardize 'Reward' here; it will be handled in standardize_df
    return df

def process_csv_file_short_or_long(file_path):
    '''
    Determines whether the file is "(short)" or "(long)" and processes it using the appropriate function.
    
    Parameters:
        file_path (Path or str): Path of the CSV file to process.
    
    Returns:
        DataFrame: The resulting DataFrame from either the short or long CSV processing function.
    '''
    file_path = Path(file_path)

    if "(short)" in file_path.stem:
        return short_csv_to_df(file_path)
    elif "(long)" in file_path.stem:
        return long_csv_to_df(file_path)
    else:
        return None

def standardize_df(df):
    '''
    Standardizes the DataFrame columns to a common set and standardizes the 'Reward' and 'Choice' column values.
    Maps columns from the '(long)' CSV files to match the standardized column names.
    Ensures that 'L' is added before 'Prob' values only when appropriate.
    
    Parameters:
        df (DataFrame): The original DataFrame.
    
    Returns:
        df_standardized (DataFrame): The standardized DataFrame.
    '''
    # Define the columns to keep and their standard names
    columns_mapping = {
        'tone': 'Tone',
        'reward': 'Reward',
        'latency': 'Latency',
        'tonetime': 'ToneTime',
        'prob': 'Prob',
        'choice': 'Choice',
        'decision': 'Choice',      # Map 'Decision' to 'Choice'
        'leftprob': 'Prob',        # Map 'Left Prob' to 'Prob'
        'left prob': 'Prob',       # Handle 'Left Prob' with space
    }

    # Standardize column names: strip spaces, make lowercase, remove spaces
    df.columns = df.columns.str.strip().str.lower().str.replace(' ', '')

    # Update columns_mapping keys to match the standardized column names
    columns_mapping = {k.replace(' ', '').lower(): v for k, v in columns_mapping.items()}

    # Keep track of which columns are in the DataFrame
    df_columns = set(df.columns)

    # Select and rename columns that exist in the DataFrame
    existing_columns = [col for col in df.columns if col in columns_mapping]
    df_standardized = df[existing_columns].rename(columns={col: columns_mapping[col] for col in existing_columns})

    # Standardize 'Reward' values
    if 'Reward' in df_standardized.columns:
        df_standardized['Reward'] = df_standardized['Reward'].astype(str).str.strip().str.lower()
        df_standardized['Reward'] = df_standardized['Reward'].replace({
            'rewarded': 'Y',
            'not rewarded': 'N',
            'y': 'Y',
            'n': 'N'
        })

    # Standardize 'Choice' values
    if 'Choice' in df_standardized.columns:
        df_standardized['Choice'] = df_standardized['Choice'].astype(str).str.strip().str.lower()
        df_standardized['Choice'] = df_standardized['Choice'].replace({
            'no decision': 'N',
            'decision r': 'R',
            'decision l': 'L',
            'r': 'R',
            'l': 'L',
            'n': 'N'
        })
        # Handle cases where 'Choice' might be 'Decision:L', 'Decision:R', etc.
        df_standardized['Choice'] = df_standardized['Choice'].apply(lambda x: x[-1].upper() if 'decision' in x else x.upper())

    # If 'Left Prob' was in the original columns, we know 'Prob' came from 'Left Prob'
    if 'leftprob' in df_columns or 'left prob' in df_columns:
        # Clean up the 'Prob' column and ensure values are strings
        df_standardized['Prob'] = df_standardized['Prob'].astype(str).str.strip()
        # Add 'L' before the integer in 'Prob' column if not already present and if the cell is not empty
        def add_L_prefix(x):
            if pd.notnull(x) and x != '':
                x = str(x).strip()
                if not x.startswith(('L', 'R')):
                    return 'L' + x
                else:
                    return x
            else:
                return x  # Return the value as is if empty or NaN

        df_standardized['Prob'] = df_standardized['Prob'].apply(add_L_prefix)
    else:
        # For 'Prob' values from '(short)' CSV files, ensure they are formatted correctly
        df_standardized['Prob'] = df_standardized['Prob'].astype(str).str.strip()

    return df_standardized

def process_csv_file_and_get_mouse_id(file_path):
    '''
    Processes a CSV file and returns a DataFrame with standardized columns, an added 'Date' column, and the mouse ID.
    
    Parameters:
        file_path (Path): Path to the CSV file.
    
    Returns:
        df (DataFrame): Processed DataFrame with standardized columns and added 'Date' column.
        mouse_id (str): The mouse ID extracted from the file name.
    '''
    df = process_csv_file_short_or_long(file_path)

    if df is None or df.empty:
        return None, None

    # Extract date code from the parent folder name
    folder_name = file_path.parent.name
    date_match = re.search(r'(\d{6})', folder_name)
    if date_match:
        date_code = date_match.group(1)
        # Use the date code as a string
        date_str = date_code
    else:
        # If no date code is found, set to an empty string or handle as needed
        date_str = ''

    # Extract mouse ID from the file name using regex
    file_name = file_path.stem
    match = re.search(r'(HPD\d+)', file_name.replace(' ', ''))
    if match:
        mouse_id = match.group(1)
    else:
        # Handle cases where 'HPD' is not found
        mouse_id = None

    # Standardize DataFrame
    df = standardize_df(df)

    # Add 'MouseID' column
    df['MouseID'] = mouse_id

    # Add 'Date' column as the last column
    df['Date'] = date_str
    # Reorder columns to ensure 'Date' is the last column
    columns_order = [col for col in df.columns if col != 'Date'] + ['Date']
    df = df[columns_order]

    return df, mouse_id

def traverse_and_collect_data(folder_path):
    '''
    Traverses folders and collects data per mouse.
    
    Parameters:
        folder_path (Path): Path to the root folder.
    
    Returns:
        mouse_data (dict): Dictionary with mouse IDs as keys and lists of DataFrames as values.
    '''
    folder = Path(folder_path)
    mouse_data = {}

    def dfs_recursive(folder):
        for item in folder.iterdir():
            if item.is_file() and item.suffix == '.csv':
                df, mouse_id = process_csv_file_and_get_mouse_id(item)

                if df is not None and mouse_id is not None:
                    if mouse_id not in mouse_data:
                        mouse_data[mouse_id] = []
                    mouse_data[mouse_id].append(df)
            elif item.is_dir():
                dfs_recursive(item)

    dfs_recursive(folder)
    return mouse_data

def write_mouse_data_to_excel(mouse_data, output_path):
    '''
    Writes mouse data to an Excel file, each sheet representing a mouse.
    
    Parameters:
        mouse_data (dict): Dictionary with mouse IDs as keys and lists of DataFrames as values.
        output_path (str or Path): Path to the output Excel file.
    '''
    with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
        for mouse_id, df_list in mouse_data.items():
            combined_df = pd.concat(df_list, ignore_index=True)
            sheet_name = mouse_id[:31]  # Excel sheet names have a maximum length of 31 characters

            combined_df.to_excel(writer, sheet_name=sheet_name, index=False)
            print(f"Written data for mouse {mouse_id} to sheet '{sheet_name}'")

In [3]:
# Set the root folder to traverse
root_folder = Path('C:/U2/Fall/COMP_396/RL_Modeling/Cohort5')  # Replace with your data folder path

# Output Excel file path
output_excel = Path('C:/U2/Fall/COMP_396/RL_Modeling/RAW_Cohort5_Compiled.xlsx')  # Replace with your desired output file path

# Traverse folders and collect data
mouse_data = traverse_and_collect_data(root_folder)

# Write data to Excel
write_mouse_data_to_excel(mouse_data, output_excel)

print("All data has been written to", output_excel)

Written data for mouse HPD14 to sheet 'HPD14'
Written data for mouse HPD15 to sheet 'HPD15'
Written data for mouse HPD17 to sheet 'HPD17'
Written data for mouse HPD18 to sheet 'HPD18'
Written data for mouse HPD19 to sheet 'HPD19'
Written data for mouse HPD20 to sheet 'HPD20'
Written data for mouse HPD21 to sheet 'HPD21'
Written data for mouse HPD22 to sheet 'HPD22'
Written data for mouse HPD23 to sheet 'HPD23'
All data has been written to C:\U2\Fall\COMP_396\RL_Modeling\RAW_Cohort5_Compiled.xlsx


In [4]:
print(mouse_data)
# for mouse_id in mouse_data.keys():
#     print(mouse_id)

'''
General Structure of mouse_data
{
    'HPD15': [
        DataFrame from 240910_HPD15_data.csv,
        DataFrame from 240911_HPD15_data.csv,
        ...
    ],
    'HPD16': [
        DataFrame from 240910_HPD16_data.csv,
        ...
    ],
    ...
}
'''

{'HPD14': [   Tone ToneTime  Latency Choice Reward  Prob MouseID    Date
0     1        0               N         L100   HPD14  240916
1     2        0               N         L100   HPD14  240916
2     3             40501      R      N  L100   HPD14  240916
3     4             55501      R      N  L100   HPD14  240916
4     5        0               N         L100   HPD14  240916
..  ...      ...      ...    ...    ...   ...     ...     ...
61   62        0               N         L100   HPD14  240916
62   63            908502      R      N  L100   HPD14  240916
63   64            921501      R      N  L100   HPD14  240916
64   65        0               N         L100   HPD14  240916
65   66        0               N         L100   HPD14  240916

[66 rows x 8 columns],      Tone  ToneTime  Prob Choice Reward Latency MouseID    Date
0       2     21009  L100      N      N           HPD14  240917
1       3     28009  L100      N      N           HPD14  240917
2       4     34009  L100    

"\nGeneral Structure of mouse_data\n{\n    'HPD15': [\n        DataFrame from 240910_HPD15_data.csv,\n        DataFrame from 240911_HPD15_data.csv,\n        ...\n    ],\n    'HPD16': [\n        DataFrame from 240910_HPD16_data.csv,\n        ...\n    ],\n    ...\n}\n"

## **1.2 Process The Compiled File**

**Creates the Data Dataframe**

In [ ]:
def process_mouse_data(mouse_data):
    """
    Processes the mouse_data dictionary into a standardized format.
    Returns a dictionary of processed DataFrames per mouse.

    Parameters:
        mouse_data (dict): Dictionary with mouse IDs as keys and lists of DataFrames as values.

    Returns:
        processed_data (dict): Dictionary with mouse IDs as keys and processed DataFrames as values.
    """
    processed_data = {}  # Initialize a dictionary to store processed DataFrames

    # Process each mouse in mouse_data
    for mouse_id, df_list in mouse_data.items():
        # Combine the list of DataFrames into one DataFrame
        df = pd.concat(df_list, ignore_index=True)

        # Standardize column names
        df.columns = df.columns.str.strip().str.lower()  # Strip spaces and make lowercase

        # Check if necessary columns exist in the input data
        required_columns = ["tone", "prob", "reward"]
        if not all(col in df.columns for col in required_columns):
            print(f"Skipping mouse '{mouse_id}' due to missing columns.")
            continue

        # Remove rows where 'tone' is empty or contains invalid values
        df["tone"] = df["tone"].astype(str).str.strip()  # Strip spaces
        df["tone"] = pd.to_numeric(df["tone"], errors="coerce")  # Convert to numeric, set invalid values to NaN
        df = df.dropna(subset=["tone"])  # Drop rows where 'tone' is NaN
        df["tone"] = df["tone"].astype(int)  # Safely convert to integers

        # Strip spaces from 'prob' and 'reward' column values
        df["prob"] = df["prob"].astype(str).str.strip()
        df["reward"] = df["reward"].astype(str).str.strip()

        # Initialize the processed DataFrame
        processed_df = pd.DataFrame()

        # Map 'Choice' values to 'Left' and 'Left Prob'
        def map_left(choice_value):
            if choice_value.startswith('L'):
                return 1
            elif choice_value.startswith('R'):
                return 0
            else:
                return None

        def map_left_prob(prob_value):
            # Map 'Prob' to numerical left probability
            if prob_value.startswith('L'):
                try:
                    prob_num = int(prob_value[1:])
                    return prob_num / 100.0
                except:
                    return None
            elif prob_value.startswith('R'):
                try:
                    prob_num = int(prob_value[1:])
                    return 1 - (prob_num / 100.0)
                except:
                    return None
            else:
                return None

        def map_reward(reward_value):
            reward_value = reward_value.lower()
            if reward_value in ["y", "rewarded", "yes"]:
                return 1
            elif reward_value in ["n", "not rewarded", "no"]:
                return 0
            else:
                return None

        # Create 'Left' column based on 'Prob'
        processed_df["Left"] = df["prob"].apply(map_left)

        # Map 'Reward' values
        processed_df["Reward"] = df["reward"].apply(map_reward)

        # Set 'Left Prob' column
        processed_df["Left Prob"] = df["prob"].apply(map_left_prob)

        # Remove rows where 'Left', 'Reward', or 'Left Prob' is None after mapping
        processed_df = processed_df[processed_df["Left"].notna()]
        processed_df = processed_df[processed_df["Reward"].notna()]
        processed_df = processed_df[processed_df["Left Prob"].notna()]

        # Add 'Session' column from the 'Date' column in the data
        if 'date' in df.columns:
            processed_df["Session"] = df['date']
        else:
            processed_df["Session"] = ''  # Or use another default value

        # Set 'Trial' column to match the 'Tone' column
        processed_df["Trial"] = df["tone"]

        # Store the processed DataFrame in the dictionary
        processed_data[mouse_id] = processed_df

    return processed_data


def write_processed_data_to_excel(processed_data, output_file):
    """
    Writes the processed data to an Excel file.

    Parameters:
        processed_data (dict): Dictionary with mouse IDs as keys and processed DataFrames as values.
        output_file (str): Path to the output Excel file.
    """
    with pd.ExcelWriter(output_file, engine="openpyxl", mode="w") as writer:
        for mouse_id, processed_df in processed_data.items():
            # Write the processed data to the output file in the corresponding sheet
            sheet_name = mouse_id[:31]  # Excel sheet names have a maximum length of 31 characters
            processed_df.to_excel(writer, sheet_name=sheet_name, index=False)
            print(f"Processed mouse: {mouse_id}")

    print(f"Output written to: {output_file}")


In [ ]:
# Example usage:

# Assuming 'mouse_data' is already defined from your previous code
# Output Excel file path
output_excel = r"C:\U2\Fall\COMP_396/RL_Modeling\Processed_Cohort5_Data.xlsx"  # Replace with your desired output file path

# Run the function
data = process_mouse_data(mouse_data)
write_processed_data_to_excel(data, output_excel)

Processed mouse: HPD14
Processed mouse: HPD15
Processed mouse: HPD17
Processed mouse: HPD18
Processed mouse: HPD19
Processed mouse: HPD20
Processed mouse: HPD21
Processed mouse: HPD22
Processed mouse: HPD23
Output written to: C:\U2\Fall\COMP_396/RL_Modeling\Processed_Cohort5_Data.xlsx


**Creates the Mouse Info Dataframe**

In [7]:
# implement this later not sure if needed rn

In [8]:

print(data['HPD14'])

       Left  Reward  Left Prob Session  Trial
2         1     0.0        1.0  240916      3
3         1     0.0        1.0  240916      4
5         1     0.0        1.0  240916      6
6         1     0.0        1.0  240916      7
8         1     0.0        1.0  240916      9
...     ...     ...        ...     ...    ...
10742     1     1.0        0.0  241003    121
10743     1     1.0        0.0  241003    122
10744     1     1.0        0.0  241003    123
10745     1     1.0        0.0  241003    124
10747     1     1.0        0.0  241003    126

[1767 rows x 5 columns]


# **2.0 RL Models**

**This section implements 10 different RL models.** Below is a table summarising the number of parameters per models and what they correspond to.


Model Name| Abbreviation | # Param| Parameters
--------  |--------------|------- |-----------
Random                                        | Random| 1|b = tendency to press left
Win-Stay/Loose-Shift                          | WSLS|1|$\varepsilon$ = tendency of win-stay/loose-shift
Q-learning                                    |Q|2|$\alpha$ = learning rate $\beta$ = exploration rate
Double Q-learning                             |DQ|3|$\alpha^+$ = learning rate for rewarded trials $\alpha^-$ = learning rate for non rewarded trials $\beta$ = exploration rate
Q learning with choice kernel                 |QCK|4|$\alpha$ = learning rate $\alpha_c$ = learning rate for choice kernel $\beta$ = exploration rate $\beta_c$ = exploration rate for choice kernel
Double Q-learning with choice kernel          |DQCK|5| $\alpha^+$ = learning rate for rewarded trials $\alpha^-$ = learning rate for non rewarded trial $\alpha_c$ = learning rate for choice kernel $\beta$ = exploration rate $\beta_c$ = exploration rate for choice kernel
Q-learning wih choice kernel balanced         |QCKe|5|$\alpha$ = learning rate $\alpha_c$ = learning rate for choice kernel $\beta$ = exploration rate $\beta_c$ = exploration rate for choice kernel $\eta$ = tendency to rely on reward history rather than action history
Double Q-learning with choice kernel balanced |DQCKe| 6 | $\alpha^+$ = learning rate for rewarded trials $\alpha^-$ = learning rate for non rewarded trials $\alpha_c$ = learning rate for choice kernel $\beta$ = exploration rate $\beta_c$ = exploration rate for choice kernel $\eta$ = tendency to rely on reward history rather than action history
Q-learning with forgetting                    |QF|3| $\alpha$ = learning rate $\alpha_f$ = forgetting rate $\beta$ = exploration rate
Double Q-learning with forgetting             |DQF| 4| $\alpha^+$ = learning rate for rewarded trials $\alpha^-$ = learning rate for non rewarded trial $\beta$ = exploration rate

In [9]:
parameter_dict = {'Random': ['b'], 'WSLS': ['ε'], 'Q':['α', 'β'],
              'DQ': ['α+', 'α-', 'β'], 'QCK': ['α', 'β', 'αc', 'β_c'],
              'DQCK': ['α+', 'α-', 'β', 'α_c', 'β_c'],
              'QCKe': ['α', 'β', 'α_c', 'β_c', 'η'],
              'DQCKe': ['α+', 'α-', 'β', 'α_c', 'β_c', 'η'],
              'QF': ['α', 'α_f', 'β'], 'DQF': ['α+', 'α-','α_f','β'], 'DQFCK':  ['α+', 'α-','α_f','β', 'αc', 'β_c']}

## **2.1 Non-RL Models**

**Random Model**

Actions are chosen randomly with a side bias $b$. The probability of choosing lever $L$ at triat $t$ is:

\begin{align}
p_t^L = b
\end{align}


In [10]:
def Random(parameters, actions, rewards, plots=False):
  b = parameters[0] #side bias
  Q = np.array([0.5, 0.5]) #Q-values, initialised at 0.5
  log_action_prob_trials =np.zeros(len(rewards)) #log probability of choosing action a at ach trial
  generated_actions = np.zeros(len(rewards))
  for t, (a,r) in enumerate(zip(actions, rewards)):
    action_prob = [1-b, b] #choose right with 1-b chance, and left with b chance
    log_action_prob_trials[t] = np.log(action_prob[a]) #save log probability of choosing the mouse action a
    generated_actions[t] = np.random.choice([0,1],p= action_prob)
  if not plots:
    return -np.sum(log_action_prob_trials)
  else:
    return generated_actions

**Win-Stay/Loose-Shift model (WSLS)**

Actions are chosen based on win-stay/loose-shift with $1- \varepsilon/2$ chance and win-shift loose-stay with $\varepsilon/2$ chance. The probability of choosing lever $L$ at trial $t$ is:


\begin{align}
        \ p_t^L = \left\{
        \begin{array}{cl}
        1- \varepsilon/2  & if & Win-Stay/Loose-Shift \\
        \varepsilon/2 & if &Win-Shift/Loose-stay
        \end{array}
        \right.
    \end{align}

In [11]:
def WSLS(parameters, actions, rewards, plots=False):
  epsilon = parameters[0] #epsilon
  Q = np.array([0.5, 0.5]) #Q-values, initialised at 0.5
  log_action_prob_trials =np.zeros(len(rewards)) #log probability of choosing action a at ach trial
  previous_a = actions[0]
  previous_r = rewards[0]
  generated_actions = np.zeros(len(rewards))
  for t, (a,r) in enumerate(zip(actions[1:], rewards[1:])):

    stay = previous_a==a

    #win-stay/loose-shift
    if stay == previous_r:
      action_prob= 1-(epsilon/2) #1-epsilon/2 prob of choosing the lever chosen by the mouse

    #win-shift/loose-stay
    else:
      action_prob= epsilon/2 #epsilon/2 prob of choosing the lever chosen by the mouse

    log_action_prob_trials[t] = np.log(action_prob) #save log probability of choosing the mouse action a
    previous_a = a
    previous_r = r



  return -np.sum(log_action_prob_trials)

## **2.2 Q-Learning Models**

**Q-Learning model (Q)**

Actions are chosen based on a standard Q-learning algorithm with one learning rate $\alpha$ and inverse temperate $\beta$. For each trial $t$, the Q-value of option $L$, $Q_{t+1}^L$, is updated as follow:

\begin{align}
Q_{t+1}^L = Q_t^L + \alpha (r_t - Q_t^L)
    \end{align}

with $r_t$ the reward (0 or 1) earned at trial $t$. Actions are then chosen by on the softmax exploration with parameter $\beta$. The probability of chosing option $L$ is as follows:

\begin{align}
p(a_{t+1} =L) = \frac{e^{\beta Q_t^L}}{\sum{e^{\beta Q_t^l}}}
\end{align}



In [12]:
def Q(parameters,*args, plots=False):
  alpha, beta = parameters
  actions, rewards = args
  #1. Initialise variables
  Q = np.zeros( (len(rewards),2) ) + 0.5 #Q values initialised as [0.5, 0.5]
  log_action_prob_trials= np.zeros(len(rewards)) #Log of the probability of choosing each action at trial t

  #2. Compute Q values at each trial t
  for t, (a,r) in enumerate(zip(actions[:-1], rewards[:-1])):
      Q[t + 1, a] = Q[t, a] + alpha * (r - Q[t, a]) #update Q value of action chosen
      Q[t + 1, 1 - a] = Q[t, 1 - a] #Q value of action non chosen stays the same

  #3.a For model fitting, return the negative log likelihood (value to minimise)
  if not plots:
    #Compute log probability of choosing each action at each trial t
    log_action_prob = log_softmax(Q*beta, axis=1) #log probability of choosing actions based on Q values
    log_action_prob = log_action_prob[np.arange(len(actions)), actions] #probability that the model would choose the action chosen by the mouse
    return -np.sum(log_action_prob)

  #3.b For plotting purposes, return probability of choosing actions at each trial t
  else:
    action_prob = softmax(Q*beta, axis=1)
    return action_prob


**Double Q-learning  model (DQ)**

Actions are chosen based on a Q-learning algorithm with two learning rate for positive and negative rewards, $\alpha^+$ and $\alpha^-$, and inverse temperate $\beta$. For each trial $t$, the Q-value of option $L$, $Q_{t+1}^L$, is updated as follow:


\begin{align}
        \ Q_{t+1}^L = \left\{
        \begin{array}{cl}
         Q_t^L + \alpha^+(r_t - Q_t^L)  & r=1 \\
         Q_t^L + \alpha^-(r_t - Q_t^L)  & r=0
        \end{array}
        \right.
    \end{align}

with $r_t$ the reward (0 or 1) earned at trial $t$. Actions are then chosen by on the softmax exploration with parameter $\beta$. The probability of chosing option $L$ is as follows:

\begin{align}
p(a_{t+1} =L) = \frac{e^{\beta Q_t^L}}{\sum{e^{\beta Q_t^l}}}
\end{align}

In [13]:
def DQ(parameters, actions, rewards,plots=False):
  alpha_pos, alpha_neg, beta = parameters

  #1. Initialise variables
  Q = np.zeros( (len(rewards),2) ) + 0.5 #Q values initialised as [0.5, 0.5]
  log_action_prob_trials= np.zeros(len(rewards)) #Log of the probability of choosing each action at trial t
  alpha = np.ones( (len(rewards),2)) #matrix with alppha_pos in column 1 and alpha_neg in columns 2
  alpha[:,0 ] =alpha[:,0 ] * alpha_neg
  alpha[:,1 ] =alpha[:,1 ] * alpha_pos
  alpha = alpha[np.arange(len(alpha)), rewards] #vector withe learning rate to use at each trial t (alpah_neg if r=0, alpha_pos if r=1)

  #2. Compute Q values at each trial t
  for t, (a,r) in enumerate(zip(actions[:-1], rewards[:-1])):
      Q[t + 1, a] = Q[t, a] + alpha[t] * (r - Q[t, a]) #update Q value of action chosen
      Q[t + 1, 1 - a] = Q[t, 1 - a] #Q value of action non chosen stays the same

  #3.a For model fitting, return the negative log likelihood (value to minimise)
  if not plots:
    #Compute log probability of choosing each action at each trial t
    log_action_prob = log_softmax(Q*beta, axis=1) #log probability of choosing actions based on Q values
    log_action_prob = log_action_prob[np.arange(len(actions)), actions] #probability that the model would choose the action chosen by the mouse
    return -np.sum(log_action_prob)

  #3.b For plotting purposes, return probability of choosing actions at each trial t
  else:
    action_prob = softmax(Q*beta, axis=1)
    return action_prob

## **2.3 Choice Kernel Models**

**Q learning+ Choice Kernel (QCK)**


This model computes a value updating function $Q$ and a choice kernel function $CK$ indepently. At each trial $t$, the Q-value of chosen lever $L$ is updated as follows:

\begin{align}
Q_{t+1}^L = Q_t^L + \alpha (r_t - Q_t^L)
    \end{align}

with $r_t$ the reward (0 or 1) earned at trial $t$. The choice kernel $CK$ of lever $L$ is then computed, and represents the tendency of a mouse to repeat the same action. It is updated as follows:

\begin{align}
CK_{t+1}^L = CK_{t}^L + \alpha_c (a_t^L -  CK_{t}^L )
    \end{align}

with $a_t^L$ a binary variable indicating whether the lever $L$ was chosen on trial $t$. If $a_t^L$=0, then $CK_{t+1}^L$ decreases, making it less likely that the mouse will choose lever $L$ at the next trial. The probability of choosing the lever $L$ is computed based on a linear combination of the $Q$ value and the choice kernel $CK$:

\begin{align}
p_t^L = \frac{e^{\beta Q_t^L +\beta_c CK_{t}^L }}{\sum{e^{\beta Q_t^l + \beta_c CK_{t}^l}}}
    \end{align}

The model has 4 parameters: learning rate for value updating \alpha, learning rate for the choice kernel updating \alpha_c, the inverse temperate for value updating \beta and for the choice kernel $\beta_c.$


In [14]:
def QCK(parameters, actions, rewards, plots=False):
  alpha, beta, alpha_c, beta_c = parameters

  #1. Initialise variables
  CK = np.zeros( (len(rewards),2)) #choice kernel values initialised as [0, 0]
  Q = np.zeros( (len(rewards),2) ) + 0.5 #Q values initialised as [0.5, 0.5]
  log_action_prob_trials= np.zeros(len(rewards)) #Log of the probability of choosing each action at trial t

  #2. Compute Q values at CK at each trial t
  for t, (a,r) in enumerate(zip(actions[:-1], rewards[:-1])):
      Q[t + 1, a] = Q[t, a] + alpha * (r - Q[t, a]) #update Q value of action chosen
      Q[t + 1, 1 - a] = Q[t, 1 - a] #Q value of action non chosen stays the same
      CK[t+1, a] = CK[t, a] + alpha_c*(1-CK[t, a] )
      CK[t+1, 1- a] = CK[t, 1- a] + alpha_c*(-CK[t, a] )

  #3.a For model fitting, return the negative log likelihood (value to minimise)
  if not plots:
    #Compute log probability of choosing each action at each trial t
    log_action_prob = log_softmax(Q*beta + CK*beta_c, axis=1) #log probability of choosing actions based on Q values
    log_action_prob = log_action_prob[np.arange(len(actions)), actions] #probability that the model would choose the action chosen by the mouse
    return -np.sum(log_action_prob)

  #3.b For plotting purposes, return probability of choosing actions at each trial t
  else:
    action_prob = softmax(Q*beta + CK*beta_c, axis=1)
    return action_prob
    return generated_actions, best_estimated_action, generated_probabilities

**double Q-learning + choice kernel (DQCK)**

This model computes a value updating function $Q$ and a choice kernel function $CK$ indepently. At each trial $t$, the Q-value of chosen lever $L$ is updated as follows:

\begin{align}
        \ Q_{t+1}^L = \left\{
        \begin{array}{cl}
         Q_t^L + \alpha^+(r_t - Q_t^L)  & r=1 \\
         Q_t^L + \alpha^-(r_t - Q_t^L)  & r=0
        \end{array}
        \right.
    \end{align}

with $r_t$ the reward (0 or 1) earned at trial $t$. The choice kernel $CK$ of lever $L$ is then computed, and represents the tendency of a mouse to repeat the same action. It is updated as follows:

\begin{align}
CK_{t+1}^L = CK_{t}^L + \alpha_c (a_t^L -  CK_{t}^L )
    \end{align}

with $a_t^L$ a binary variable indicating whether the lever $L$ was chosen on trial $t$. If $a_t^L$=0, then $CK_{t+1}^L$ decreases, making it less likely that the mouse will choose lever $L$ at the next trial. The probability of choosing the lever $L$ is computed based on a linear combination of the $Q$ value and the choice kernel $CK$:

\begin{align}
p_t^L = \frac{e^{\beta Q_t^L +\beta_c CK_{t}^L }}{\sum{e^{\beta Q_t^l + \beta_c CK_{t}^l}}}
    \end{align}

The model has 4 parameters: learning rate for value updating \alpha, learning rate for the choice kernel updating $\alpha_c$, the inverse temperate for value updating \beta and for the choice kernel $\beta_c.$




In [15]:
def DQCK(parameters, actions, rewards, plots=False):
  alpha_pos, alpha_neg, beta, alpha_c, beta_c = parameters

  #1. Initialise variables
  CK = np.zeros( (len(rewards),2)) #choice kernel values initialised as [0, 0]
  Q = np.zeros( (len(rewards),2) ) + 0.5 #Q values initialised as [0.5, 0.5]
  log_action_prob_trials= np.zeros(len(rewards)) #Log of the probability of choosing each action at trial t
  alpha = np.ones( (len(rewards),2)) #matrix with alppha_pos in column 1 and alpha_neg in columns 2
  alpha[:,0 ] =alpha[:,0 ] * alpha_neg
  alpha[:,1 ] =alpha[:,1 ] * alpha_pos
  alpha = alpha[np.arange(len(alpha)), rewards] #vector withe learning rate to use at each trial t (alpah_neg if r=0, alpha_pos if r=1)

  #2. Compute Q values at CK at each trial t
  for t, (a,r) in enumerate(zip(actions[:-1], rewards[:-1])):
      Q[t + 1, a] = Q[t, a] + alpha[t] * (r - Q[t, a]) #update Q value of action chosen
      Q[t + 1, 1 - a] = Q[t, 1 - a] #Q value of action non chosen stays the same
      CK[t+1, a] = CK[t, a] + alpha_c*(1-CK[t, a] )
      CK[t+1, 1- a] = CK[t, 1- a] + alpha_c*(-CK[t, a] )

  #3.a For model fitting, return the negative log likelihood (value to minimise)
  if not plots:
    #Compute log probability of choosing each action at each trial t
    log_action_prob = log_softmax(Q*beta + CK*beta_c, axis=1) #log probability of choosing actions based on Q values
    log_action_prob = log_action_prob[np.arange(len(actions)), actions] #probability that the model would choose the action chosen by the mouse
    return -np.sum(log_action_prob)

  #3.b For plotting purposes, return probability of choosing actions at each trial t
  else:
    action_prob = softmax(Q*beta + CK*beta_c, axis=1)
    return action_prob

**Q-learning and choice kernel balanced (QCKe)**

This model is similar to the previous ones and computes a value updating function $Q$ and a choice kernel function $CK$ indepently.It adds a parameter $\eta$  that tunes the balance between the value updating rule and the choice kernel updating rule. This allows animals to depend their choices more heavily on values or on choice preference. At each trial $t$, the Q-value of chosen lever $L$ is updated as follows:

\begin{align}
        Q_{t+1}^L = Q_t^L + \alpha (r_t - Q_t^L)
    \end{align}


with $r_t$ the reward (0 or 1) earned at trial $t$. The choice kernel $CK$ of lever $L$ is then computed, and represents the tendency of a mouse to repeat the same action. It is updated as follows:

\begin{align}
CK_{t+1}^L = CK_{t}^L + \alpha_c (a_t^L -  CK_{t}^L )
    \end{align}

with $a_t^L$ a binary variable indicating whether the lever $L$ was chosen on trial $t$. If $a_t^L$=0, then $CK_{t+1}^L$ decreases, making it less likely that the mouse will choose lever $L$ at the next trial. The probability of choosing the lever $L$ is computed based on a linear combination of the $Q$ value and the choice kernel $CK$:

\begin{align}
p_t^L = \frac{e^{\eta \beta Q_t^L +(1-\eta)\beta_c CK_{t}^L }}{\sum{e^{\eta\beta Q_t^l + (1-\eta)\beta_c CK_{t}^l}}}
    \end{align}

The model has 4 parameters: learning rate for value updating $\alpha$, learning rate for the choice kernel updating $\alpha_c$, the inverse temperate for value updating $\beta$ and for the choice kernel $\beta_c.$


In [16]:
def QCKe(parameters, actions, rewards, plots=False):
  alpha, beta, alpha_c, beta_c, eta = parameters
  #1. Initialise variables
  CK = np.zeros( (len(rewards),2)) #choice kernel values initialised as [0, 0]
  Q = np.zeros( (len(rewards),2) ) + 0.5 #Q values initialised as [0.5, 0.5]
  log_action_prob_trials= np.zeros(len(rewards)) #Log of the probability of choosing each action at trial t

  #2. Compute Q values at CK at each trial t
  for t, (a,r) in enumerate(zip(actions[:-1], rewards[:-1])):
      Q[t + 1, a] = Q[t, a] + alpha * (r - Q[t, a]) #update Q value of action chosen
      Q[t + 1, 1 - a] = Q[t, 1 - a] #Q value of action non chosen stays the same
      CK[t+1, a] = CK[t, a] + alpha_c*(1-CK[t, a] )
      CK[t+1, 1- a] = CK[t, 1- a] + alpha_c*(-CK[t, a] )

  #3.a For model fitting, return the negative log likelihood (value to minimise)
  if not plots:
    #Compute log probability of choosing each action at each trial t
    log_action_prob = log_softmax(eta*Q*beta + (1-eta)*CK*beta_c, axis=1) #log probability of choosing actions based on Q values
    log_action_prob = log_action_prob[np.arange(len(actions)), actions] #probability that the model would choose the action chosen by the mouse
    return -np.sum(log_action_prob)

  #3.b For plotting purposes, return probability of choosing actions at each trial t
  else:
    action_prob = softmax(eta*Q*beta + (1-eta)*CK*beta_c, axis=1)
    return action_prob

**double Qlearning and choice kernel balanced (DQCKe)**

Same but with two learning rates for positive and negative outcomes.

In [17]:
def DQCKe(parameters, actions, rewards, plots=False):
  alpha_pos, alpha_neg, beta, alpha_c, beta_c, eta = parameters

   #1. Initialise variables
  CK = np.zeros( (len(rewards),2)) #choice kernel values initialised as [0, 0]
  Q = np.zeros( (len(rewards),2) ) + 0.5 #Q values initialised as [0.5, 0.5]
  log_action_prob_trials= np.zeros(len(rewards)) #Log of the probability of choosing each action at trial t
  alpha = np.ones( (len(rewards),2)) #matrix with alppha_pos in column 1 and alpha_neg in columns 2
  alpha[:,0 ] =alpha[:,0 ] * alpha_neg
  alpha[:,1 ] =alpha[:,1 ] * alpha_pos
  alpha = alpha[np.arange(len(alpha)), rewards] #vector withe learning rate to use at each trial t (alpah_neg if r=0, alpha_pos if r=1)

  #2. Compute Q values at CK at each trial t
  for t, (a,r) in enumerate(zip(actions[:-1], rewards[:-1])):
      Q[t + 1, a] = Q[t, a] + alpha[t] * (r - Q[t, a]) #update Q value of action chosen
      Q[t + 1, 1 - a] = Q[t, 1 - a] #Q value of action non chosen stays the same
      CK[t+1, a] = CK[t, a] + alpha_c*(1-CK[t, a] )
      CK[t+1, 1- a] = CK[t, 1- a] + alpha_c*(-CK[t, a] )

  #3.a For model fitting, return the negative log likelihood (value to minimise)
  if not plots:
    #Compute log probability of choosing each action at each trial t
    log_action_prob = log_softmax(eta*Q*beta + (1-eta)*CK*beta_c, axis=1) #log probability of choosing actions based on Q values
    log_action_prob = log_action_prob[np.arange(len(actions)), actions] #probability that the model would choose the action chosen by the mouse
    return -np.sum(log_action_prob)

  #3.b For plotting purposes, return probability of choosing actions at each trial t
  else:
    action_prob = softmax(eta*Q*beta + (1-eta)*CK*beta_c, axis=1)
    return action_prob

## **2.4 Forgetting Models**

**Q-learning with forgetting** (Ito & Doya, 2009)

This model assumes that if a lever is not chosen at trial $t$, the mouse will 'forget' the value of that lever, and the Q-value slowly decreases with learning rate $\alpha^f$. This is equivalent to standard Q-learning if $\alpha^f=0$ (no forgetting). At each trial $t$, the Q-value of lever $L$ is updated as follows:

\begin{align}
        \ Q_{t+1}^L = \left\{
        \begin{array}{cl}
         Q_t^L + \alpha(r_t - Q_t^L)  & if & a_{t}=L\\
         Q_t^L + \alpha^f(- Q_t^L)  & if & a_{t}\neq L \\
        \end{array}
        \right.
    \end{align}

with $a_t$ boolean variable indicating whether the lever $L$ was chosen during trial $t$.

In [18]:
def QF(parameters, actions, rewards, plots=False):
  alpha, alpha_f, beta = parameters

  #1. Initialise variables
  Q = np.zeros( (len(rewards),2) ) + 0.5 #Q values initialised as [0.5, 0.5]
  log_action_prob_trials= np.zeros(len(rewards)) #Log of the probability of choosing each action at trial t

  #2. Compute Q values at each trial t
  for t, (a,r) in enumerate(zip(actions[:-1], rewards[:-1])):
      Q[t + 1, a] = Q[t, a] + alpha * (r - Q[t, a]) #update Q value of action chosen
      Q[t + 1, 1 - a] = Q[t, 1 - a] - alpha_f*Q[t, 1 - a] #Q value of action non chosen decreases over time

  #3.a For model fitting, return the negative log likelihood (value to minimise)
  if not plots:
    #Compute log probability of choosing each action at each trial t
    log_action_prob = log_softmax(Q*beta, axis=1) #log probability of choosing actions based on Q values
    log_action_prob = log_action_prob[np.arange(len(actions)), actions] #probability that the model would choose the action chosen by the mouse
    return -np.sum(log_action_prob)

  #3.b For plotting purposes, return probability of choosing actions at each trial t
  else:
    action_prob = softmax(Q*beta, axis=1)
    return action_prob


**double Q-learning with forgetting** (Ito & Doya, 2009)

Same as Q-learning with forgetting, but with assymetric learning rates for reinforced & non reinforced trials.

\begin{align}
        \ Q_{t+1}^L = \left\{
        \begin{array}{cl}
         Q_t^L + \alpha^+(r_t - Q_t^L)  & if & a_{t}=L, r=1\\
         Q_t^L + \alpha^-(r_t - Q_t^L)  & if & a_{t}=L, r=0\\
         Q_t^L + \alpha^f(- Q_t^L)  & if & a_{t}\neq L \\
        \end{array}
        \right.
    \end{align}

with $a_t$ boolean variable indicating whether the lever $L$ was chosen during trial $t$.

In [19]:
def DQF(parameters, actions, rewards, plots=False):
  alpha_pos, alpha_neg, alpha_f, beta = parameters

  #1. Initialise variables
  Q = np.zeros( (len(rewards),2) ) + 0.5 #Q values initialised as [0.5, 0.5]
  log_action_prob_trials= np.zeros(len(rewards)) #Log of the probability of choosing each action at trial t
  alpha = np.ones( (len(rewards),2)) #matrix with alppha_pos in column 1 and alpha_neg in columns 2
  alpha[:,0 ] =alpha[:,0 ] * alpha_neg
  alpha[:,1 ] =alpha[:,1 ] * alpha_pos
  alpha = alpha[np.arange(len(alpha)), rewards] #vector withe learning rate to use at each trial t (alpah_neg if r=0, alpha_pos if r=1)

  #2. Compute Q values at each trial t
  for t, (a,r) in enumerate(zip(actions[:-1], rewards[:-1])):
      Q[t + 1, a] = Q[t, a] + alpha[t] * (r - Q[t, a]) #update Q value of action chosen
      Q[t + 1, 1 - a] = Q[t, 1 - a] - alpha_f*Q[t, 1 - a] #Q value of action non chosen decreases over time

  #3.a For model fitting, return the negative log likelihood (value to minimise)
  if not plots:
    #Compute log probability of choosing each action at each trial t
    log_action_prob = log_softmax(Q*beta, axis=1) #log probability of choosing actions based on Q values
    log_action_prob = log_action_prob[np.arange(len(actions)), actions] #probability that the model would choose the action chosen by the mouse
    return -np.sum(log_action_prob)

  #3.b For plotting purposes, return probability of choosing actions at each trial t
  else:
    action_prob = softmax(Q*beta, axis=1)
    return action_prob

**double Q-learning with forgetting & choice kernel**

Combines the assymetry in learning rates (double Q learning, $ \alpha^+$ and $\alpha^-$ ) with the idea that the values of non-chosen actions are forgetted (forggeting , $\alpha^f$) and that the mice have an overall tendency to repeat actions (choice kernel, $ \alpha_c$, $\beta_c$)


**Value updating rule:**

\begin{align}
        \ Q_{t+1}^L = \left\{
        \begin{array}{cl}
         Q_t^L + \alpha^+(r_t - Q_t^L)  & if & a_{t}=L, r=1\\
         Q_t^L + \alpha^-(r_t - Q_t^L)  & if & a_{t}=L, r=0\\
         Q_t^L + \alpha^f(- Q_t^L)  & if & a_{t}\neq L \\
        \end{array}
        \right.
    \end{align}

**Choice kernel updating rule:**

\begin{align}
CK_{t+1}^L = CK_{t}^L + \alpha_c (a_t^L -  CK_{t}^L )
    \end{align}

**Policy:**

\begin{align}
p_t^L = \frac{e^{ \beta Q_t^L +\beta_c CK_{t}^L }}{\sum{e^{\beta Q_t^l + \beta_c CK_{t}^l}}}
    \end{align}


In [20]:
def DQFCK(parameters, actions, rewards, plots=False):
  alpha_pos, alpha_neg, alpha_f, beta, alpha_c, beta_c = parameters

  #1. Initialise variables
  CK = np.zeros( (len(rewards),2)) #choice kernel values initialised as [0, 0]
  Q = np.zeros( (len(rewards),2) ) + 0.5 #Q values initialised as [0.5, 0.5]
  log_action_prob_trials= np.zeros(len(rewards)) #Log of the probability of choosing each action at trial t
  alpha = np.ones( (len(rewards),2)) #matrix with alppha_pos in column 1 and alpha_neg in columns 2
  alpha[:,0 ] =alpha[:,0 ] * alpha_neg
  alpha[:,1 ] =alpha[:,1 ] * alpha_pos
  alpha = alpha[np.arange(len(alpha)), rewards] #vector withe learning rate to use at each trial t (alpah_neg if r=0, alpha_pos if r=1)

  #2. Compute Q values at CK at each trial t
  for t, (a,r) in enumerate(zip(actions[:-1], rewards[:-1])):
      Q[t + 1, a] = Q[t, a] + alpha[t] * (r - Q[t, a]) #update Q value of action chosen
      Q[t + 1, 1 - a] = Q[t, 1 - a] - alpha_f*Q[t, 1 - a] #Q value of action non chosen decreases over time
      CK[t+1, a] = CK[t, a] + alpha_c*(1-CK[t, a] )
      CK[t+1, 1- a] = CK[t, 1- a] + alpha_c*(-CK[t, a] )

  #3.a For model fitting, return the negative log likelihood (value to minimise)
  if not plots:
    #Compute log probability of choosing each action at each trial t
    log_action_prob = log_softmax(Q*beta + CK*beta_c, axis=1) #log probability of choosing actions based on Q values
    log_action_prob = log_action_prob[np.arange(len(actions)), actions] #probability that the model would choose the action chosen by the mouse
    return -np.sum(log_action_prob)

  #3.b For plotting purposes, return probability of choosing actions at each trial t
  else:
    action_prob = softmax(Q*beta + CK*beta_c, axis=1)
    return action_prob

# **3.0 Model Fitting Functions**

Get AIC value

In [21]:
def AIC(k, negloglik):
  return(2*k -2*(-negloglik))

Use scipy.optimise.minimise ind the set of parameter that best explain the data. This is a gradient descent-like algorithm that find the parameters that minimise the log likelihood P(y | parameters).


In [22]:
def minimise_loglik(start, actions, rewards, fun=Random, parameter_dict= parameter_dict):

  #Here we use a gradient-descent with constraints. For example, by definition, our learning rates are bounded by 0 and 1.
  #Note that if the results of your algorithm spits out value close to the bounds (alpha=1 or alpha=0), this may indicate problem of convergence.
  bounds_constraints_dict = { 'α':  (0, 1), 'β': (0, np.inf), 'η': (0,1), 'ε': (0.000001,0.99999), 'b': (0.000001,0.99999) }

  #Get the parameters associated with the model
  params = parameter_dict[fun.__name__]

  #get the bounds associated with the parameters
  bnds = []
  for p in params:
    bnds += [bounds_constraints_dict[p[0]]]

  #minimise negative log likelihood
  output = minimize(fun=fun, x0=start, args=(actions, rewards), method="L-BFGS-B", bounds=bnds)

  return output

Minimise the log likelihood using n random initial values. When using a gradient-descent algorithm, there is always a risk of getting stuck in a local optimum. To get around that, we perform a gradient descent multiple times with different values and use the results with the smallest loss.

In [23]:
def random_initial_minimise_loglik(actions, rewards, fun=Random, n_starts=5, parameter_dict= parameter_dict):

  results =[]
  parameters = []
  start=[]


  random_starts_dict = { 'α':  np.random.random, 'β': np.random.randint, 'η': np.random.random, 'ε': np.random.random, 'b': np.random.random}
  #get parameters associated with the model
  model_param = parameter_dict[fun.__name__]

  #define random starts
  random_starts = []
  for n in range(n_starts):
    starts = []
    for p in model_param:
      if p[0] == 'β' :
       s =random_starts_dict[p[0]](low=0, high=100)/10
       starts.append(s)
      else:
        s =random_starts_dict[p[0]]()
        starts.append(s)
    random_starts.append(starts)

  #fit models using each random initial start
  for s in random_starts:
      output = minimise_loglik(s, actions, rewards, fun=fun)
      #print(output)
      results.append(output.fun)
      parameters.append(output.x)

  #take the results from the random start that gave the best results
  best_start = np.argmin(results)
  best_log_lik = results[best_start]
  best_param = parameters[best_start]
  return best_log_lik, best_param

Fit on one mouse data

In [24]:
def model_fit_all_sessions(mouse_data, models = [Random, WSLS, Q, DQ, QCK, DQCK, QCKe, DQCKe, QF, DQF, DQFCK]):

  results = pd.DataFrame([])
  parameters = []
  log_liks = []
  sessions = []
  model_names=[]
  AICs = []

  actions = mouse_data['Left'].to_list()
  rewards = mouse_data['Reward'].to_list()

  #2.fit models
  for model in models:
      log_lik, param = random_initial_minimise_loglik(actions, rewards, fun=model, n_starts=5) #get the negative log likelihood and the best set of parameters
      sessions.append(1)
      parameters.append(param)
      log_liks.append(log_lik)
      model_names.append(model.__name__)
      AICs.append(AIC(len(param)+1, log_lik)) #get AIC (add+1 to parameters for the intercept)

  #compute AIC weights
  deltaAIC = np.array(AICs) - np.min(np.array(AICs))
  exp = np.sum(np.exp(-0.5*deltaAIC))
  AIC_weights =  np.exp(-0.5*deltaAIC)/exp

  #create a dataframe to store all the results
  results = pd.DataFrame([sessions, model_names, log_liks, parameters, AICs, AIC_weights])
  results =results.T
  results.columns = ['Session', 'Model', 'Log Lik', 'Parameters', 'AIC', 'AIC_weights']

  return results

Fit on all mice

In [25]:
def fit(data=data, mouse_info=Mouse_Info, models = [Random, WSLS, Q, DQ, QCK, DQCK, QCKe, DQCKe, QF, DQF, DQFCK]):
  all_mice_results = pd.DataFrame([])
  x =0
  m=0
  for mouse in mouse_info['ID'].to_list():
    m+=1
    #fit all models
    values = data[mouse] #get data
    results = model_fit_all_sessions(values, models=models) #fit all models on the data
    print('Mouse ', m, '/', len ( mouse_info), ' done.')

    #get info about the mouse
    info = []
    group_name = []
    for group in Mouse_Info.columns:
      if group != 'ID':
        info += (mouse_info.loc[mouse_info['ID']==mouse][group].to_list())
        group_name.append(group)

    #add results to the all mice results dataframe
    results = results.T
    for col in results.columns:
        all_mice_results= pd.concat([all_mice_results, pd.DataFrame([mouse]+results[col].to_list()+ info)], axis=1)
        x=x+1

  all_mice_results = all_mice_results.T
  all_mice_results.columns = ['ID', 'Session', 'Model', 'LogLike', 'Param', 'AIC', 'AIC weights' ]+ group_name

  return all_mice_results

NameError: name 'Mouse_Info' is not defined